In [23]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import pyspark as ps
import json


In [24]:
!pwd

/home/jovyan/work/galvanize/capstone1/src


In [25]:
for filename in os.listdir('/home/jovyan/work/galvanize/capstone1/src'):
    print(type(filename))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [26]:

with open('../data/competitions.json') as data_file:    
    data = json.load(data_file)
competitions_df = pd.json_normalize(data, sep = "_")

In [27]:
competitions_df.head()

,competition_id,season_id,country_name,competition_name,competition_gender,season_name,match_updated,match_available
0,16,4,Europe,Champions League,male,2018/2019,2020-07-29T05:00,2020-07-29T05:00
1,16,1,Europe,Champions League,male,2017/2018,2020-07-29T05:00,2020-07-29T05:00
2,16,2,Europe,Champions League,male,2016/2017,2020-08-26T12:33:15.869622,2020-07-29T05:00
3,16,27,Europe,Champions League,male,2015/2016,2020-08-26T12:33:15.869622,2020-07-29T05:00
4,16,26,Europe,Champions League,male,2014/2015,2020-08-26T12:33:15.869622,2020-07-29T05:00


In [28]:
competitions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   competition_id      37 non-null     int64 
 1   season_id           37 non-null     int64 
 2   country_name        37 non-null     object
 3   competition_name    37 non-null     object
 4   competition_gender  37 non-null     object
 5   season_name         37 non-null     object
 6   match_updated       37 non-null     object
 7   match_available     37 non-null     object
dtypes: int64(2), object(6)
memory usage: 2.4+ KB


In [29]:
with open('../data/events/15946.json') as data_file:    
    data = json.load(data_file)
event15946_df = pd.json_normalize(data, sep = "_")

In [30]:
event15946_df.head()

,id,index,period,timestamp,minute,second,possession,duration,type_id,type_name,...,substitution_outcome_name,substitution_replacement_id,substitution_replacement_name,pass_cut_back,shot_one_on_one,foul_committed_advantage,foul_won_advantage,foul_committed_type_id,foul_committed_type_name,pass_goal_assist
0,ce508a95-38d3-4248-a50e-dc8d7e23230c,1,1,00:00:00.000,0,0,1,0.000,35,Starting XI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,39abd1c2-2575-41be-ae79-3e936eead529,2,1,00:00:00.000,0,0,1,0.000,35,Starting XI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1148e967-380f-421f-97f0-d0c820c72984,3,1,00:00:00.000,0,0,1,0.000,18,Half Start,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7cf6876d-7c50-4d64-b999-e2a039641c81,4,1,00:00:00.000,0,0,1,0.000,18,Half Start,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,34208ade-2af4-45c3-970e-655937cad938,5,1,00:00:00.098,0,0,2,1.909,30,Pass,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
event15946_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4002 entries, 0 to 4001
Columns: 105 entries, id to pass_goal_assist
dtypes: float64(32), int64(9), object(64)
memory usage: 3.2+ MB


In [32]:
data = json.load(open('../data/lineups/15946.json'))
lineup15946_df = pd.json_normalize(data, sep = "_")

In [33]:
lineup15946_df.head()

,team_id,team_name,lineup
0,217,Barcelona,"[{'player_id': 3109, 'player_name': 'Malcom Fi..."
1,206,Deportivo Alavés,"[{'player_id': 6566, 'player_name': 'Borja Gon..."


In [34]:
lineup15946_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   team_id    2 non-null      int64 
 1   team_name  2 non-null      object
 2   lineup     2 non-null      object
dtypes: int64(1), object(2)
memory usage: 176.0+ bytes


In [35]:
#need to determine what league and season i am in first

In [36]:
# for filename in os.listdir('../data/events'):
#     with open(f'../data/matches/{filename}') as data_file:    
#         data = json.load(data_file)
#     match15946_df = pd.json_normalize(data, sep = "_")

In [38]:
#run json reader loop into spark, so they load quicker, than convert to pandas and use json_normalize. from here i can go back to spark or stick with pandas

spark = (ps.sql.SparkSession.builder 
        .master("local[4]") 
        .appName("sparkSQL exercise") 
        .getOrCreate()
        )
sc = spark.sparkContext

In [100]:
#make list of match ids and zip
def match_ids(json_path,folder_name):
    ids = []
    for filename in os.listdir(json_path):
        ids.append(folder_name + filename[:-5])
    return ids
    
def json_folder_reader(json_path,match_ids):
    lst = []
    for filename,idx in zip(os.listdir(json_path),match_ids):
        if '.json' not in filename:
            continue
        var = pd.json_normalize(json.load(open(json_path + '/' + filename)),sep="_")
        print(json_path)
        print(filename)
        lst.append(var)
        return lst

In [101]:
json_folder_reader('../data/events','events')

../data/events
15946.json


[                                        id  index  period     timestamp  \
 0     ce508a95-38d3-4248-a50e-dc8d7e23230c      1       1  00:00:00.000   
 1     39abd1c2-2575-41be-ae79-3e936eead529      2       1  00:00:00.000   
 2     1148e967-380f-421f-97f0-d0c820c72984      3       1  00:00:00.000   
 3     7cf6876d-7c50-4d64-b999-e2a039641c81      4       1  00:00:00.000   
 4     34208ade-2af4-45c3-970e-655937cad938      5       1  00:00:00.098   
 ...                                    ...    ...     ...           ...   
 3997  be8a2725-6a34-45d5-8a49-5c9e0208e805   3998       2  00:46:41.244   
 3998  55d71847-9511-4417-aea9-6f415e279011   3999       2  00:46:42.770   
 3999  f93a1612-e2de-4efe-b202-6c4a178eebad   4000       2  00:46:43.268   
 4000  e1cc4d5e-ba55-4b6b-88cc-dae13311c1d9   4001       2  00:47:32.053   
 4001  cd767526-5f64-4600-acbf-e0a2608f0d00   4002       2  00:47:32.053   
 
       minute  second  possession  duration  type_id    type_name  ...  \
 0          

In [83]:
# filename = pd.json_normalize(data, sep = "_")

In [84]:
# lineups_dict = json_folder_reader('../data/lineups')

In [89]:
lineups_match_ids = match_ids('../data/lineups','lineups')

lineups_match_ids[:5]

['lineups15946',
 'lineups15956',
 'lineups15973',
 'lineups15978',
 'lineups15986']

In [90]:
lineups_sparkdf_lst = json_folder_reader('../data/lineups',lineups_match_ids)
lineups_sparkdf_lst[:5]

TypeError: 'NoneType' object is not subscriptable

In [87]:
for i in lineups_sparkdf_lst:
    i = i.toPandas()
for i in lineups_sparkdf_lst:
    i = pd.json_normalize(i,sep = "_")

TypeError: 'NoneType' object is not iterable

In [ ]:
data = json.load(open('../data/lineups/15946.json'))
lineup15946_df = pd.json_normalize(data, sep = "_")

In [91]:
dct = {}
for i in lineups_match_ids:
    d[i:8] = pd.json_normalize(json.load(open(json_path + '/' + filename)), sep = "_")
    

NameError: name 'json_path' is not defined

In [ ]:
#use pandas instead, loop through json normalization and wait for it
